# Testing Tight Divergence on CIFAR-10

In [1]:
import sys
sys.path.append("../..")

## Train a MLP on MNIST

In [2]:
import perceptrons.cifar10 as cifar10

cifar10_mlp = cifar10.CIFAR10(
    architecture    = [3072, 1024, 512, 128, 10],
    num_epochs      = 20,
    batch_size      = 128,
    name            = "CIFAR10-mlp"
)

Epoch 1/20
313/313 [==============================] - 24s 75ms/step - loss: 100.1105 - accuracy: 0.1255 - val_loss: 94.8839 - val_accuracy: 0.1236
Epoch 2/20
313/313 [==============================] - 24s 76ms/step - loss: 93.9555 - accuracy: 0.1252 - val_loss: 89.4103 - val_accuracy: 0.1231
Epoch 3/20
313/313 [==============================] - 24s 75ms/step - loss: 88.3134 - accuracy: 0.1260 - val_loss: 84.4803 - val_accuracy: 0.1243
Epoch 4/20
313/313 [==============================] - 24s 76ms/step - loss: 83.6791 - accuracy: 0.1254 - val_loss: 79.8880 - val_accuracy: 0.1241
Epoch 5/20
313/313 [==============================] - 24s 76ms/step - loss: 79.0887 - accuracy: 0.1269 - val_loss: 75.4957 - val_accuracy: 0.1226
Epoch 6/20
313/313 [==============================] - 24s 75ms/step - loss: 74.9916 - accuracy: 0.1258 - val_loss: 71.2394 - val_accuracy: 0.1231
Epoch 7/20
313/313 [==============================] - 24s 76ms/step - loss: 70.4942 - accuracy: 0.1267 - val_loss: 67.0638 

KeyboardInterrupt: 

In [ ]:
mnist_mlp.report()

## Create Tight Convex Approximation

In [ ]:
## Create the domain interval
import numpy as np
import geometry.interval as interval

InputDomain = interval.Interval(
    np.zeros(mnist_mlp.in_shape),
    np.ones(mnist_mlp.in_shape)
)

## Create convex approximation
import perceptrons.convex as convex

conv_mnist = convex.ConvexApprox(
    mnist_mlp,
    InputDomain
)
conv_mnist.report()

## Test on Random Inputs

In [ ]:
## Get random points
N               = 1000
RandomInputs    = InputDomain.random_points(N)

We test some random inputs, of the following form:

In [ ]:
## Sample Random Input
import matplotlib.pyplot as plt
plt.imshow(RandomInputs[0], cmap="Greys", vmin=0.0, vmax=1.0)
plt.colorbar()
plt.show()

### Average Divergence

In [ ]:
relu_scores = mnist_mlp.scores(RandomInputs)
conv_scores = conv_mnist.scores(RandomInputs)

tight_divergence        = np.max(np.abs(relu_scores - conv_scores), 0)
avg_tight_divergence    = np.sum(tight_divergence) / len(tight_divergence)

print("Avg. Tight Divergence:", avg_tight_divergence)

### Worst Case Lower Bound

In [ ]:
wc_lb_relu_score            = mnist_mlp.scores(np.zeros(mnist_mlp.in_shape))
wc_lb_conv_score            = conv_mnist.scores(np.zeros(mnist_mlp.in_shape))
diff                        = wc_lb_relu_score - wc_lb_conv_score
worst_lb_tight_divergence   = np.max(np.abs(diff))

print("Worst Case Lower Bound:", worst_lb_tight_divergence)

### Worst Case Upper Bound

In [ ]:
conv_range_ub               = conv_mnist.propagate_bounds(InputDomain)[0][-1].ub
worst_ub_tight_divergence   = np.max(np.abs(conv_range_ub))

print("Worst Case Upper Bound:", worst_ub_tight_divergence)

### Counting Misclassifications

In [ ]:
relu_preds              = mnist_mlp.predict(RandomInputs)
conv_preds              = conv_mnist.predict(RandomInputs)
misclassifications      = np.array(relu_preds) != np.array(conv_preds)
misclassification_prob  = np.sum(misclassifications) / len(misclassifications)

print("Probability of Misclassification of the Tight Conv. Approximation:", misclassification_prob)

## Test on Test-Set Inputs

In [ ]:
## Get random points
N                   = 1000
RandomTestInputs    = mnist_mlp.random_points(N)

We test some random Test-Set inputs, of the following form:

In [ ]:
plt.imshow(RandomTestInputs[0], cmap="Greys", vmin=0.0, vmax=1.0)
plt.colorbar()
plt.show()

In [ ]:
Sample              = RandomTestInputs[0]
sample_relu_score   = mnist_mlp.scores(Sample)
sample_conv_score   = conv_mnist.scores(Sample)
sample_relu_predict = mnist_mlp.predict(Sample)
sample_conv_predict = conv_mnist.predict(Sample)

sample_diff                = sample_relu_score - sample_conv_score
sample_tight_divergence    = np.max(np.abs(diff))

print("ReLU Sample score:\n", sample_relu_score)
print("Conv Sample score:\n", sample_conv_score)
print("Scores Tight Divergence:", sample_tight_divergence)
print("ReLU Sample Prediction:", sample_relu_predict[0])
print("Conv Sample Prediction:", sample_conv_predict[0])

### Average Divergence

In [ ]:
test_relu_scores = mnist_mlp.scores(RandomTestInputs)
test_conv_scores = conv_mnist.scores(RandomTestInputs)

test_tight_divergence        = np.max(np.abs(test_relu_scores - test_conv_scores), 0)
test_avg_tight_divergence    = np.sum(test_tight_divergence) / len(test_tight_divergence)

print("Avg. Tight Divergence:", test_avg_tight_divergence)

### Counting Misclassifications

In [ ]:
test_relu_preds              = mnist_mlp.predict(RandomInputs)
test_conv_preds              = conv_mnist.predict(RandomInputs)
test_misclassifications      = np.array(test_relu_preds) != np.array(test_conv_preds)
test_misclassification_prob  = np.sum(test_misclassifications) / len(test_misclassifications)

print("Probability of Misclassification of the Tight Conv. Approximation:", test_misclassification_prob)